In [1]:
import pandas as pd

def prepare_detection_dataset(buggy_file, fixed_file, output_csv):
    with open(buggy_file, 'r', encoding='utf-8') as bf, open(fixed_file, 'r', encoding='utf-8') as ff:
        buggy_codes = [line.strip() for line in bf]
        fixed_codes = [line.strip() for line in ff]

    # Create a dataset
    data = []
    for buggy in buggy_codes:
        data.append({'code': buggy, 'label': 1})  # Buggy: 1
    for fixed in fixed_codes:
        data.append({'code': fixed, 'label': 0})  # Fixed: 0

    # Save to CSV
    df = pd.DataFrame(data)
    df.to_csv(output_csv, index=False)
    print(f"✅ Detection dataset created and saved at: {output_csv}")

# Example usage:
prepare_detection_dataset(
    buggy_file='/content/train.buggy-fixed.buggy',
    fixed_file='/content/train.buggy-fixed.fixed',
    output_csv='/content/detection_dataset.csv'
)


✅ Detection dataset created and saved at: /content/detection_dataset.csv


In [9]:
import pandas as pd

def prepare_detection_dataset(buggy_file, fixed_file, output_csv):
    with open(buggy_file, 'r', encoding='utf-8') as bf, open(fixed_file, 'r', encoding='utf-8') as ff:
        buggy_codes = [line.strip() for line in bf]
        fixed_codes = [line.strip() for line in ff]

    # Create a dataset
    data = []
    for buggy in buggy_codes:
        data.append({'code': buggy, 'label': 1})  # Buggy: 1
    for fixed in fixed_codes:
        data.append({'code': fixed, 'label': 0})  # Fixed: 0

    # Save to CSV
    df = pd.DataFrame(data)
    df.to_csv(output_csv, index=False)
    print(f"✅ Detection dataset created and saved at: {output_csv}")

# Example usage:
prepare_detection_dataset(
    buggy_file="/content/valid.buggy-fixed.buggy",
    fixed_file="/content/valid.buggy-fixed.fixed",
    output_csv='/content/detection_datasetval.csv'
)


✅ Detection dataset created and saved at: /content/detection_datasetval.csv


In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [4]:
from transformers import AutoTokenizer
from datasets import load_dataset

# Load CodeT5 tokenizer
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-small")  # or use your fine-tuned one if needed

# Load your CSV as Huggingface Dataset
dataset = load_dataset("csv", data_files="detection_dataset.csv")

def tokenize_function(examples):
    return tokenizer(
        examples["code"],  # 'code' is the column with code snippets
        padding="max_length",
        truncation=True,
        max_length=512,
    )

# Tokenize
tokenized_dataset = dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

Map:   0%|          | 0/104728 [00:00<?, ? examples/s]

In [11]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['code', 'label', 'input_ids', 'attention_mask'],
        num_rows: 104728
    })
})

In [12]:
tokenized_dataset_val

DatasetDict({
    train: Dataset({
        features: ['code', 'label', 'input_ids', 'attention_mask'],
        num_rows: 13092
    })
})

In [10]:
# Load your CSV as Huggingface Dataset
dataset = load_dataset("csv", data_files="/content/detection_datasetval.csv")

def tokenize_function(examples):
    return tokenizer(
        examples["code"],  # 'code' is the column with code snippets
        padding="max_length",
        truncation=True,
        max_length=512,
    )

# Tokenize
tokenized_dataset_val = dataset.map(tokenize_function, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/13092 [00:00<?, ? examples/s]

In [5]:
from transformers import AutoModelForSequenceClassification

# Load CodeT5 model for classification
model = AutoModelForSequenceClassification.from_pretrained(
    "Salesforce/codet5-small",
    num_labels=2  # 2 classes: buggy (1) and fixed (0)
)


config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at Salesforce/codet5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# QLoRA config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "v"],  # if you want to specify; or leave default
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"  # Sequence Classification
)

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# Apply LoRA
model = get_peft_model(model, lora_config)

# Check trainable params
model.print_trainable_parameters()


trainable params: 589,824 || all params: 61,345,794 || trainable%: 0.9615


In [15]:
from transformers import Trainer, TrainingArguments



training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",   # evaluate every epoch
    save_strategy="epoch",          # save checkpoint every epoch
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,             # keep only last 2 checkpoints
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    report_to="none"                # (disable wandb/huggingface logging if you don't use them)
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset_val["train"],  # or separate val split if you have
    tokenizer=tokenizer,
)

# Train 🚀
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-15-00a7d55d9a89>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,0.699800,0.691609


KeyError: "The `metric_for_best_model` training argument is set to 'eval_accuracy', which is not found in the evaluation metrics. The available evaluation metrics are: ['eval_loss']. Consider changing the `metric_for_best_model` via the TrainingArguments."

In [16]:
# Save the model and tokenizer
trainer.save_model("./saved_codet5_detection_model")
tokenizer.save_pretrained("./saved_codet5_detection_model")


('./saved_codet5_detection_model/tokenizer_config.json',
 './saved_codet5_detection_model/special_tokens_map.json',
 './saved_codet5_detection_model/vocab.json',
 './saved_codet5_detection_model/merges.txt',
 './saved_codet5_detection_model/added_tokens.json',
 './saved_codet5_detection_model/tokenizer.json')

In [17]:
from huggingface_hub import notebook_login

notebook_login()  # Logs you in



In [18]:
repo_name = "qlora-codet5-java-bugdetection"


model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

adapter_model.safetensors:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nairabhiram907/qlora-codet5-java-bugdetection/commit/af9ee7a1951ca7caef1be7b85e401de409c3dbf7', commit_message='Upload tokenizer', commit_description='', oid='af9ee7a1951ca7caef1be7b85e401de409c3dbf7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nairabhiram907/qlora-codet5-java-bugdetection', endpoint='https://huggingface.co', repo_type='model', repo_id='nairabhiram907/qlora-codet5-java-bugdetection'), pr_revision=None, pr_num=None)

In [20]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the saved model
model_path = "/content/saved_codet5_detection_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()


Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at Salesforce/codet5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5ForSequenceClassification(
  (transformer): T5Model(
    (shared): Embedding(32100, 512)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32100, 512)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): lora.Linear(
                  (base_layer): Linear(in_features=512, out_features=512, bias=False)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=512, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=16, out_features=512, bias=False)
                  )
                  (lora_embedding_A): ParameterDict()
                  (lora_embedding_B): ParameterDict()
                  (lora_magnitude_

In [21]:
def detect_bug(code_snippet):
    inputs = tokenizer(code_snippet, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        prediction = torch.argmax(probs, dim=1).item()

    label = "Buggy" if prediction == 1 else "Fixed"
    confidence = probs[0][prediction].item()

    return label, round(confidence, 4)


In [22]:
# Example code snippet
sample_code = "public static int add(int a, int b) { return a - b; }"

label, confidence = detect_bug(sample_code)
print(f"Prediction: {label} (Confidence: {confidence})")


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Prediction: Fixed (Confidence: 1.0)
